<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/GNN_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.0 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 52.0 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.7 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-gbzc84l_
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-gbzc84l_
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 3f4f1a008c479b16e30456ee3d9ef8410a3909be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.6.0-py3-none-any.whl size=11

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import files

# Load dataset
uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)
df = df.sample(frac=0.4, random_state=42)
# Graph Preparation
tweets_column = 'tweet'
labels_column = 'label'
NUM_LABELS = len(df[labels_column].unique())
possible_labels = df[labels_column].unique()
label_dict = {possible_label: index for index, possible_label in enumerate(possible_labels)}
df['labels'] = df[labels_column].map(label_dict)

# Split the dataset
df_labeled, df_temp = train_test_split(df, stratify=df[labels_column], test_size=0.8)
df_unlabeled, df_test = train_test_split(df_temp, stratify=df_temp[labels_column], test_size=0.25)


Saving Arabic_Depression_10.000_Tweets.xlsx to Arabic_Depression_10.000_Tweets.xlsx


In [ ]:

# Correct the graph creation function to have proper feature size
def create_graph_data(df, feature_dim=64):
    num_nodes = len(df)

    # Create random features for nodes as an example (Replace with actual features if available)
    x = torch.randn(num_nodes, feature_dim, dtype=torch.float)  # Node features with 64 dimensions
    labels = torch.tensor(df['labels'].values, dtype=torch.long)

    # Sample graph: Replace this with your logic for creating edges between nodes
    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index, y=labels)
    return data

# Create graph data with corrected feature dimensions
graph_data_labeled = create_graph_data(df_labeled, feature_dim=64)
graph_data_unlabeled = create_graph_data(df_unlabeled, feature_dim=64)
graph_data_test = create_graph_data(df_test, feature_dim=64)

from torch_geometric.data import Data
import torch
import random

def split_data(data, num_subgraphs=10, min_nodes_per_subgraph=10):
    """
    Splits a large graph data object into multiple smaller subgraphs.

    Args:
        data (Data): The original graph data object containing node features, edges, and labels.
        num_subgraphs (int): The number of subgraphs to create.
        min_nodes_per_subgraph (int): Minimum number of nodes per subgraph.

    Returns:
        List[Data]: A list of smaller graph data objects (subgraphs).
    """
    node_indices = list(range(data.num_nodes))
    random.shuffle(node_indices)  # Randomize node indices for sampling

    subgraphs = []
    nodes_per_subgraph = max(min_nodes_per_subgraph, len(node_indices) // num_subgraphs)

    for i in range(0, len(node_indices), nodes_per_subgraph):
        subgraph_node_indices = node_indices[i:i + nodes_per_subgraph]
        subgraph_node_indices = torch.tensor(subgraph_node_indices, dtype=torch.long)

        # Extract features and labels for the subgraph nodes
        subgraph_x = data.x[subgraph_node_indices]
        subgraph_y = data.y[subgraph_node_indices]

        # Filter edges that connect the nodes within this subgraph
        mask = torch.isin(data.edge_index[0], subgraph_node_indices) & torch.isin(data.edge_index[1], subgraph_node_indices)
        subgraph_edge_index = data.edge_index[:, mask]

        # Reindex edges for the new subgraph
        node_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(subgraph_node_indices.tolist())}
        subgraph_edge_index = torch.tensor(
            [[node_mapping[src.item()], node_mapping[dst.item()]] for src, dst in subgraph_edge_index.t()],
            dtype=torch.long
        ).t().contiguous()

        # Create subgraph Data object
        subgraph = Data(x=subgraph_x, edge_index=subgraph_edge_index, y=subgraph_y)
        subgraphs.append(subgraph)

    return subgraphs

# Example usage:
subgraphs_labeled = split_data(graph_data_labeled, num_subgraphs=10, min_nodes_per_subgraph=10)
subgraphs_unlabeled = split_data(graph_data_unlabeled, num_subgraphs=10, min_nodes_per_subgraph=10)
subgraphs_test = split_data(graph_data_test, num_subgraphs=10, min_nodes_per_subgraph=10)


In [ ]:
batch_size = 4
# DataLoaders for batched subgraphs
dataloader_train = DataLoader(subgraphs_labeled, batch_size=batch_size, shuffle=True)
dataloader_unlabeled = DataLoader(subgraphs_unlabeled, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(subgraphs_test, batch_size=batch_size, shuffle=False)

# Define GNN Model with correct input dimensions
class GNNModel(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the models with the corrected input feature size
student_model = GNNModel(num_node_features=64, hidden_channels=64, num_classes=NUM_LABELS)
teacher_model = GNNModel(num_node_features=64, hidden_channels=64, num_classes=NUM_LABELS)

# Set up the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student_model.to(device)
teacher_model.to(device)

# Copy student model parameters to the teacher model
teacher_model.load_state_dict(student_model.state_dict())

# Set up optimizer
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
epochs = 10
alpha = 0.999  # EMA decay rate

# Define evaluation metrics
def compute_metrics(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accuracy = accuracy_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat, average='weighted')
    precision = precision_score(labels_flat, preds_flat, average='weighted')
    recall = recall_score(labels_flat, preds_flat, average='weighted')
    return accuracy, f1, precision, recall

# Update teacher model using EMA of student model
def update_teacher(student_model, teacher_model, alpha):
    for student_param, teacher_param in zip(student_model.parameters(), teacher_model.parameters()):
        teacher_param.data = alpha * teacher_param.data + (1 - alpha) * student_param.data

# Training loop
for epoch in range(1, epochs + 1):
    student_model.train()
    teacher_model.eval()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc=f'Epoch {epoch}', leave=False, disable=False)

    # Train the student on labeled data
    for batch in progress_bar:
        student_model.zero_grad()
        batch = batch.to(device)

        # Forward pass through the student model
        logits_student = student_model(batch.x, batch.edge_index)
        loss = F.cross_entropy(logits_student, batch.y)  # Supervised loss on labeled data

        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})

    # Consistency Loss on Unlabeled Data
    loss_consistency_total = 0
    for batch in DataLoader([graph_data_unlabeled], batch_size=batch_size, shuffle=True):
        student_model.zero_grad()
        batch = batch.to(device)

        # Predictions from student and teacher models
        student_preds = student_model(batch.x, batch.edge_index)
        teacher_preds = teacher_model(batch.x, batch.edge_index)

        # Compute consistency loss (e.g., Mean Squared Error between predictions)
        consistency_loss = F.mse_loss(student_preds, teacher_preds.detach())
        loss_consistency_total += consistency_loss.item()

        # Backward pass and optimization
        consistency_loss.backward()
        torch.nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Update the teacher model using EMA
    update_teacher(student_model, teacher_model, alpha)

    loss_train_avg = loss_train_total / len(dataloader_train)
    loss_consistency_avg = loss_consistency_total / len(dataloader_unlabeled)

    tqdm.write(f'\nEpoch {epoch}')
    tqdm.write(f'Supervised Training Loss: {loss_train_avg}')
    tqdm.write(f'Consistency Loss: {loss_consistency_avg}')

# Evaluation on test data
teacher_model.eval()
loss_test_total = 0
predictions, true_vals = [], []

for batch in tqdm(dataloader_test, desc='Testing', leave=False):
    batch = batch.to(device)
    with torch.no_grad():
        outputs = teacher_model(batch.x, batch.edge_index)
    loss = F.cross_entropy(outputs, batch.y)
    loss_test_total += loss.item()

    preds = outputs.detach().cpu().numpy()
    labels = batch.y.cpu().numpy()

    predictions.append(preds)
    true_vals.append(labels)

predictions = np.concatenate(predictions, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

# Calculate metrics
test_accuracy, test_f1, test_precision, test_recall = compute_metrics(predictions, true_vals)
print(f'Testing Accuracy: {test_accuracy}')
print(f'Testing F1 Score: {test_f1}')
print(f'Testing Precision: {test_precision}')
print(f'Testing Recall: {test_recall}')


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)



Epoch 1
Supervised Training Loss: 0.6934611201286316
Consistency Loss: 6.336216756608337e-06



Epoch 2
Supervised Training Loss: 0.6909687320391337
Consistency Loss: 5.8091530566647025e-06



Epoch 3
Supervised Training Loss: 0.6909323533376058
Consistency Loss: 3.6625384988534884e-06



Epoch 4
Supervised Training Loss: 0.6899540821711222
Consistency Loss: 3.4506432105748295e-06


...


Epoch 7
Supervised Training Loss: 0.6893983880678812
Consistency Loss: 2.578878896504951e-06



Epoch 8
Supervised Training Loss: 0.6897861758867899
Consistency Loss: 2.4809872532690256e-06



Epoch 9
Supervised Training Loss: 0.6901172200838724
Consistency Loss: 2.390316543217826e-06



Epoch 10
Supervised Training Loss: 0.6893367568651835
Consistency Loss: 2.3675609857794675e-06


Testing Accuracy: 0.49375
Testing F1 Score: 0.4726307962715244
Testing Precision: 0.4865438988095238
Testing Recall: 0.49375
